In [ ]:
CPD_URL = "<Enter your CPD endpint here>"
CPD_USERNAME = "<username>"
CPD_PASSWORD = "<password>"

CPD_URL = CPD_URL.rstrip("/")

WML_CREDENTIALS = {
    "url": CPD_URL,
    "password": CPD_PASSWORD,
    "username": CPD_USERNAME,
    "instance_id": "wml_local",
    "version": "4.5"
}

SPACE_ID = "<provide a space id here>"

ROWS_TO_SCORE = 10

In [ ]:
from ibm_watson_machine_learning import APIClient

wml_client = APIClient(WML_CREDENTIALS)
print(wml_client.version)

In [ ]:
wml_client.set.default_space(SPACE_ID)

In [ ]:
wml_client.deployments.list()

In [ ]:
MODEL_UID = "03f33cc8-f4d3-43e2-b586-98c0ec5c9f16"

In [ ]:
import itc_utils.flight_service as itcfs

readClient = itcfs.get_flight_client()

nb_data_request = {
    'data_name': """modeling_records_2022.csv""",
    'interaction_properties': {
        #'row_limit': 500,
        'infer_schema': 'false',
        'infer_as_varchar': 'false'
    }
}

flightInfo = itcfs.get_flight_info(readClient, nb_data_request=nb_data_request)

data_df_1 = itcfs.read_pandas_and_concat(readClient, flightInfo, timeout=240)
data_df_1.head(10)

In [ ]:
import pandas as pd

In [ ]:
scoring_df = data_df_1.sample(n=ROWS_TO_SCORE).drop(['ATTRITION'], axis=1)
scoring_df = scoring_df.apply(pd.to_numeric, errors='coerce').fillna(scoring_df)

scoring_df.head()

In [ ]:
scoring_df.dtypes

In [ ]:
fields = scoring_df.columns.tolist()
values = scoring_df[fields].values.tolist()

In [ ]:
payload = {"input_data": [{"fields": fields, "values": []}]}

In [ ]:
import math

for row in values:
    new_row = []
    for value in row:
        if value == math.floor(value):
            new_row.append(math.floor(value))
        else:
            new_row.append(value)
    payload["input_data"][0]["values"].append(new_row)

In [ ]:
response = wml_client.deployments.score(MODEL_UID, payload)

In [ ]:
response